In [ ]:
'''!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
'''

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-g_o2kge7/kobert-tokenizer_52421acdc25e4e4390923313b0b7a9cb
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-g_o2kge7/kobert-tokenizer_52421acdc25e4e4390923313b0b7a9cb
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=e085f3b2204a1c2b2a437c61149cce5135b5ae0166d782a04d778a144f8f6a35
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qjy2x2w/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!git clone https://github.com/e9t/nsmc

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torchvision.transforms as transforms
from torchvision import datasets


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-large")

#vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
device = torch.device("cuda:0")

In [ ]:
import pandas as pd

df = pd.read_csv('mbti_100_4_ns.csv',encoding='UTF-8')
df = df.drop('Unnamed: 0',axis = 1)

df

In [ ]:
len(df)

In [ ]:
df = df.dropna(axis=0)

len(df)

In [ ]:
'''
df.loc[(df['mbti'] == "intj"), 'mbti'] = 0
df.loc[(df['mbti'] == "intp"), 'mbti'] = 1
df.loc[(df['mbti'] == "entj"), 'mbti'] = 2
df.loc[(df['mbti'] == "entp"), 'mbti'] = 3
df.loc[(df['mbti'] == "infj"), 'mbti'] = 4
df.loc[(df['mbti'] == "infp"), 'mbti'] = 5
df.loc[(df['mbti'] == "enfj"), 'mbti'] = 6
df.loc[(df['mbti'] == "enfp"), 'mbti'] = 7
df.loc[(df['mbti'] == "istj"), 'mbti'] = 8
df.loc[(df['mbti'] == "istp"), 'mbti'] = 9
df.loc[(df['mbti'] == "estj"), 'mbti'] = 10
df.loc[(df['mbti'] == "estp"), 'mbti'] = 11
df.loc[(df['mbti'] == "isfj"), 'mbti'] = 12
df.loc[(df['mbti'] == "isfp"), 'mbti'] = 13
df.loc[(df['mbti'] == "esfj"), 'mbti'] = 14
df.loc[(df['mbti'] == "esfp"), 'mbti'] = 15
'''
df.loc[(df['ns'] == "n"), 'ns'] = 0
df.loc[(df['ns'] == "s"), 'ns'] = 1


In [ ]:
df

In [ ]:
data_list = []
for q, label in zip(df['text'], df['ns'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

data_list

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.2, random_state=0)

In [ ]:
!pip install -q transformers pytorch_lightning emoji soynlp

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [ ]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-large'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size
    batch_size: int = 0  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size`
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    train_data: list =  X_train # Train Dataset file
    val_data: list = X_test  # Validation Dataset file
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

In [ ]:
args.train_data
a = torch.tensor(args.train_data['text'].to_list(), dtype=torch.long)
a

In [ ]:
!nvidia-smi

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.arg = options
        self.bert = BertForSequenceClassification.from_pretrained(self.arg.pretrained_model)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.arg.pretrained_tokenizer
            if self.arg.pretrained_tokenizer
            else self.arg.pretrained_model
        )

    def forward(self, **kwargs):
        return self.bert(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.arg.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def on_validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.arg.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.arg.lr)
        elif self.arg.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.arg.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.arg.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.arg.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def preprocess_dataframe(self, df):
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

        def clean(x):
            x = pattern.sub(' ', x)
            x = url_pattern.sub('', x)
            x = x.strip()
            x = repeat_normalize(x, num_repeats=2)
            return x

        df['text'] = df['text'].map(lambda x: self.tokenizer.encode(
            clean(str(x)),
            padding='max_length',
            max_length=self.arg.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = self.arg.train_data
        df = self.preprocess_dataframe(df)


        dataset = TensorDataset(
            torch.tensor(df['text'].to_list(), dtype=torch.long),
            torch.tensor(df['ns'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.arg.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.arg.cpu_workers,
        )

    def val_dataloader(self):
        df = self.arg.val_data
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['text'].to_list(), dtype=torch.long),
            torch.tensor(df['ns'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.arg.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.arg.cpu_workers,
        )


In [ ]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 16  # Force setup batch_size

In [ ]:
print("Using PyTorch Ver", torch.__version__)
print("Fix Seed:", args.random_seed)
seed_everything(args.random_seed)
model = Model(args)

print(":: Start Training ::")

trainer = Trainer(
        epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        # For GPU Setup
        deterministic=torch.cuda.is_available(),
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        # tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
trainer.fit(model)